In [ ]:
%load_ext autoreload
%cd /workdir/ComfyUI/
%autoreload 2
from pathlib import Path
import numpy as np
import torch
import matplotlib.pyplot as plt
from comfywr.csd_lib import load_checkpoints, load_image, image_scale_by, cn_preprocess, control_net_set_create, control_net_set_apply_hint, clip_set_last_layer, image_scale
from comfywr.my_lib import animate_images, images_to_video, read_video, make_unrolled_video, revert_unrolled_video
from comfywr.my_test import interpolate_prompts_adaptive, postprocess_interpolated, interpolate_img2img_adaptive, hq_infer_txt, video_transform
from comfywr.my_lib import iter_subplots_axes

In [ ]:
!ls ./models/checkpoints/

In [ ]:
!ls ./models/upscale_models/

In [ ]:
checkpoints_names = dict(
    sd='dreamshaper_8.safetensors',
    upscale_model='RealESRGAN_x4plus_anime_6B.pth',
    # cn_depth='control_v11f1p_sd15_depth.pth',
    # cn_openpose='control_v11p_sd15_openpose.pth',
    # cn_lineart='control_v11p_sd15_lineart.pth',
    # cn_normal='control_v11p_sd15_normalbae.pth',
    # cn_canny='control_v11p_sd15_canny.pth',
    # cn_scribble='control_v11p_sd15_scribble.pth',
    # cn_mediapipe_face='control_v2p_sd15_mediapipe_face.safetensors',
    # cn_qr='controlnetQRPatternQR_v2Sd15.safetensorsx',
)
checkpoints = load_checkpoints({k:v for k,v in checkpoints_names.items()})

In [ ]:
# check if you have EasyNegative.safetensors used in example below
!ls ./models/embeddings/

In [ ]:
n_images = 2
sampler_settings = dict(sampler_name='dpmpp_2m_sde_gpu', scheduler='karras', seed=0,
                        cfg=8.0, steps=26, batch_index=np.arange(n_images))

pos = [
    'best quality',
    'masterpiece concept art',
    'a fantasy medieval marketplace',
    'complex details',
]
neg = 'embedding:EasyNegative.safetensors'
pos = '(' + ') ('.join(pos) + ')'

dim = (1024, 576)
img = hq_infer_txt(checkpoints, *dim, n_images, pos, neg,
                   sampler_settings=sampler_settings,
                   upscale_by=1.6,
                   use_upscaler=True,
                   clip_skip=2)

animate_images(postprocess_interpolated(img))